In [4]:
import json
import re
import time
from vllm import LLM, SamplingParams

# 读取JSON文件
def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# 解析output_blocks中特定类型的条目
def parse_blocks(output_blocks, type):
    type_to_block = {}
    pattern = rf"{type} (\d+)"  # 假设类型后的数字仍有用，例如标识ID或序号
    for block in output_blocks:
        match = re.search(pattern, block)
        if match:
            identifier = int(match.group(1))  # 获取类型后的数字
            type_to_block[identifier] = block
    return type_to_block

# 生成检查内容的prompt
def create_prompts(checks, type_to_block):
    prompts = []
    for identifier, event_desc in checks.items():
        identifier = int(identifier)  # 确保转换为整数
        if identifier in type_to_block:
            prompts.append(type_to_block[identifier] + f" Does this description include the {event_desc}? Please answer with 'yes' or 'no' only.")
    return prompts

# 主函数

file_path = "/home/yuhao/THREADING-THE-NEEDLE/Evalution/results.json"
datas = read_json(file_path)
prompts_once = []
prompts_range = []
prompts_periodic = []
for data in datas:
    checks_block = parse_blocks(data['output_blocks'], data['type'])
    # 生成once, range, periodic的prompts
    prompts_once.extend(create_prompts(data['checks_once'], checks_block))
    prompts_range.extend(create_prompts(data['checks_range'], checks_block))
    prompts_periodic.extend(create_prompts(data['checks_periodic'], checks_block))

    # 在此处可以添加代码处理prompts，例如发送到模型或存储结果



In [5]:
prompts_range

[" Week 13 (March 26th - April 1st)\nI'm excited to finally get away from it all and take a break. This week, I'm heading to Cambodia to explore the ancient temples. It's going to be a fantastic adventure, and I'm looking forward to immersing myself in the culture and history.\n\n Does this description include the Exploring the ancient temples of Cambodia? Please answer with 'yes' or 'no' only.",
 " Week 14 (April 2nd - April 8th)\nI'm still in Cambodia, and it's been an incredible experience so far. I've seen some breathtaking temples and learned so much about the history and culture of the area. It's been a real privilege to be here. Nothing much else happened this week.\n\n Does this description include the Exploring the ancient temples of Cambodia? Please answer with 'yes' or 'no' only.",
 " Week 11 (March 12th - March 18th)\nExcitement this week as we embarked on an adventure to explore the ancient temples of Cambodia! It was an incredible experience, and I'm so grateful for the o

In [1]:


import time
from vllm import LLM, SamplingParams
import torch
import json

# Define the sampling parameters
sampling_params = SamplingParams(temperature=0.95, top_p=0.95, max_tokens=50, seed=42)

# Example lists of prompts


# Record the start time
start_time = time.time()

# Initialize the LLM with the specified model and configuration
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct", tensor_parallel_size=2)

def evaluate_accuracy(prompts, llm, sampling_params):
    # Generate responses using the LLM
    outputs = llm.generate(prompts, sampling_params)
    total = len(outputs)
    correct = sum([1 for output in outputs if 'yes' in output.outputs[0].text])
    return correct / total if total > 0 else 0

# Evaluate the accuracy for each set of prompts
acc_once = evaluate_accuracy(prompts_once, llm, sampling_params)
acc_range = evaluate_accuracy(prompts_range, llm, sampling_params)
acc_periodic = evaluate_accuracy(prompts_periodic, llm, sampling_params)

# Print the accuracy results
print("Accuracy for once:", acc_once)
print("Accuracy for range:", acc_range)
print("Accuracy for periodic:", acc_periodic)

# Print the elapsed time
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-08-02 20:08:03,769	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-02 20:08:04 config.py:715] Defaulting to use mp for distributed inference
WARNING 08-02 20:08:04 arg_utils.py:762] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-02 20:08:04 config.py:806] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-02 20:08:04 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.11it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.85it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.66it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.63it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.45it/s]



INFO 08-02 20:08:09 model_runner.py:692] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=3944603) INFO 08-02 20:08:09 model_runner.py:692] Loading model weights took 7.5122 GB
INFO 08-02 20:08:10 distributed_gpu_executor.py:56] # GPU blocks: 62561, # CPU blocks: 4096
(VllmWorkerProcess pid=3944603) ERROR 08-02 20:08:10 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method initialize_cache: CUDA out of memory. Tried to allocate 1.91 GiB. GPU  has a total capacity of 79.15 GiB of which 1.70 GiB is free. Process 3944009 has 8.71 GiB memory in use. Including non-PyTorch memory, this process has 68.71 GiB memory in use. Of the allocated memory 66.76 GiB is allocated by PyTorch, and 25.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cu

 Week 28 (July 9th - July 15th)
Surrounded by thriving scenarios amidst identical approaches of life lies beneath spring, relaxes midst dedicated fountain. With every moment spend in our city celebrated person whose expertise knows absolutely – nostalgia I needed thinking foundations, thoughts basically, navigating. Oh what why dramatically soft bang self-relevance caught sure prosperous instal clearer morning confidence dances upon watched discovery increasing attain pebbled rock reserve – discovered time while awaits organ palms compassion all into start at urge tongue grateful guardians bad planting.

Does this description include the child_1 birthday? Please answer with 'yes' or 'no' only.
 Week 28 (July 9th - July 15th)
Surrounded by thriving scenarios amidst identical approaches of life lies beneath spring, relaxes midst dedicated fountain. With every moment spend in our city celebrated person whose expertise knows absolutely – nostalgia I needed thinking foundations, thoughts ba

In [ ]:
import json
import re
import time
import pandas as pd
from vllm import LLM, SamplingParams

# 读取JSON文件
def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# 写入JSON文件
def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# 解析output_blocks中特定类型的条目
def parse_blocks(output_blocks, type):
    type_to_block = {}
    pattern = rf"{type} (\d+)"  # 假设类型后的数字仍有用，例如标识ID或序号
    for block in output_blocks:
        match = re.search(pattern, block)
        if match:
            identifier = int(match.group(1))  # 获取类型后的数字
            type_to_block[identifier] = block
    return type_to_block

# 生成检查内容的prompt
def create_prompts(checks, type_to_block):
    prompts = []
    identifiers = []
    for identifier, event_desc in checks.items():
        identifier = int(identifier)  # 确保转换为整数
        if identifier in type_to_block:
            prompts.append(type_to_block[identifier] + f" Does this description include the {event_desc}? Please answer with 'yes' or 'no' only.")
            identifiers.append(identifier)
    return prompts, identifiers

# 定义评估准确性的函数
def evaluate_accuracy(prompts, llm, sampling_params):
    outputs = llm.generate(prompts, sampling_params)
    results = []
    for output in outputs:
        response = output.outputs[0].text.strip().lower()
        result = 'yes' if 'yes' in response else 'no'
        results.append(result)
    return results

# 保存准确率到CSV文件
def save_accuracy_to_csv(file_path, model_name, completion_rate, acc_once, acc_range, acc_periodic):
    df = pd.DataFrame({
        'Model': [model_name],
        'Completion Rate': [completion_rate],
        'Accuracy Once': [acc_once],
        'Accuracy Range': [acc_range],
        'Accuracy Periodic': [acc_periodic],
        'Average Accuracy': [(acc_once + acc_range + acc_periodic) / 3]
    })
    
    try:
        existing_df = pd.read_csv(file_path)
        existing_df = existing_df[existing_df['Model'] != model_name]  # 删除相同模型名称的行
        df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        pass
    
    df.to_csv(file_path, index=False)

# 计算完成度
def calculate_completion_rate(type_to_block, total_number):
    identifiers = set(type_to_block.keys())
    expected_identifiers = set(range(1, total_number + 1))
    missing_identifiers = expected_identifiers - identifiers
    completion_rate = (len(expected_identifiers) - len(missing_identifiers)) / len(expected_identifiers)
    return completion_rate * 100

# 主函数
json_file_path = "/home/yuhao/THREADING-THE-NEEDLE/Evalution/results/Meta-Llama-3.1-8B-Instruct_maxlen8000.json"
csv_file_path = "/home/yuhao/THREADING-THE-NEEDLE/Evalution/results/accuracy_results.csv"
model_name = json_file_path.split('/')[-1].replace('.json', '')

datas = read_json(json_file_path)

prompts_once = []
prompts_range = []
prompts_periodic = []
identifiers_once = []
identifiers_range = []
identifiers_periodic = []

completion_rate = 0
for data in datas:
    checks_block = parse_blocks(data['output_blocks'], data['type'])
    # 生成once, range, periodic的prompts
    p_once, ids_once = create_prompts(data['checks_once'], checks_block)
    p_range, ids_range = create_prompts(data['checks_range'], checks_block)
    p_periodic, ids_periodic = create_prompts(data['checks_periodic'], checks_block)
    
    prompts_once.extend(p_once)
    identifiers_once.extend(ids_once)
    
    prompts_range.extend(p_range)
    identifiers_range.extend(ids_range)
    
    prompts_periodic.extend(p_periodic)
    identifiers_periodic.extend(ids_periodic)

    data['count_once'] = len(ids_once)
    data['count_range'] = len(ids_range)
    data['count_periodic'] = len(ids_periodic)

    
    # 计算完成度
    completion_rate += calculate_completion_rate(checks_block, data['number'])

completion_rate /= len(datas)  # 平均完成度

# Define the sampling parameters
sampling_params = SamplingParams(temperature=0.95, top_p=0.95, max_tokens=50, seed=42)

# Record the start time
start_time = time.time()

# Initialize the LLM with the specified model and configuration
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct", tensor_parallel_size=2)

# Evaluate the accuracy for each set of prompts
results_once = evaluate_accuracy(prompts_once, llm, sampling_params)
results_range = evaluate_accuracy(prompts_range, llm, sampling_params)
results_periodic = evaluate_accuracy(prompts_periodic, llm, sampling_params)

# 计算准确率
acc_once = sum(1 for result in results_once if result == 'yes') / len(results_once) if results_once else 0
acc_range = sum(1 for result in results_range if result == 'yes') / len(results_range) if results_range else 0
acc_periodic = sum(1 for result in results_periodic if result == 'yes') / len(results_periodic) if results_periodic else 0

# 将结果添加到JSON文件中
start_index_once = 0
start_index_range = 0
start_index_periodic = 0
for data in datas:
    data['results_once'] = {str(identifiers_once[i]): results_once[i] for i in range(start_index_once, start_index_once + data['count_once'])}
    start_index_once += data['count_once']
    
    data['results_range'] = {str(identifiers_range[i]): results_range[i] for i in range(start_index_range, start_index_range + data['count_range'])}
    start_index_range += data['count_range']
    
    data['results_periodic'] = {str(identifiers_periodic[i]): results_periodic[i] for i in range(start_index_periodic, start_index_periodic + data['count_periodic'])}
    start_index_periodic += data['count_periodic']

# 写回JSON文件
write_json(json_file_path, datas)

# 保存准确率到CSV文件
save_accuracy_to_csv(csv_file_path, model_name, completion_rate, acc_once, acc_range, acc_periodic)

# Print the elapsed time
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
